In [132]:
import yfinance as yf
import datetime
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models, expected_returns

In [133]:
import warnings

# Disable all warnings
warnings.filterwarnings("ignore")

In [134]:
# Define the list of stock tickers
tickers = ['AAPL', 'AMD', 'AMZN', 'CCJ', 'COST', 'GOOG', 'GS', 'JPM', 'LLY', 'META', 'MSFT', 'NEE', 'PFE', 'SAP', 'WMT']
measurement_days = 10
remaining_days = 30-measurement_days

In [135]:
# Get 5 year adjusted close prices for all stocks

stock_prices = {}
short_window_prices = {}
future_prices = {}

for ticker in tickers:
    data = yf.download(ticker,period="5y")
    print(ticker)
    stock_prices[ticker] = data['Adj Close'].tolist()
    stock_data = stock_prices[ticker]
    prices_x = [stock_data[i:i+measurement_days] for i in range(len(stock_data)-remaining_days)]
    prices_y = [stock_data[i+measurement_days] for i in range(measurement_days,len(stock_data)-measurement_days)]

    prices_test = [stock_data[i:i+measurement_days] for i in range(len(stock_data)-remaining_days,len(stock_data)-measurement_days)]

    short_window_prices[ticker] = (prices_x, prices_y, prices_test)

    rf = RandomForestRegressor()
    rf.fit(prices_x,prices_y)
    # Predict stock price for 10 future days
    future_prices[ticker] = rf.predict(prices_test)



[*********************100%***********************]  1 of 1 completed
AAPL
[*********************100%***********************]  1 of 1 completed
AMD
[*********************100%***********************]  1 of 1 completed
AMZN
[*********************100%***********************]  1 of 1 completed
CCJ
[*********************100%***********************]  1 of 1 completed
COST
[*********************100%***********************]  1 of 1 completed
GOOG
[*********************100%***********************]  1 of 1 completed
GS
[*********************100%***********************]  1 of 1 completed
JPM
[*********************100%***********************]  1 of 1 completed
LLY
[*********************100%***********************]  1 of 1 completed
META
[*********************100%***********************]  1 of 1 completed
MSFT
[*********************100%***********************]  1 of 1 completed
NEE
[*********************100%***********************]  1 of 1 completed
PFE
[*********************100%********************

In [136]:
future_prices = pd.DataFrame(future_prices)
print(future_prices)

# Construct covariance matrix of future stock prices
# cov_matrix = risk_models.sample_cov(future_prices)
S = risk_models.CovarianceShrinkage(future_prices).ledoit_wolf()
# plotting.plot_covariance(S, plot_correlation=True)

# Use capm to find expected returns on future prices
mu = expected_returns.capm_return(future_prices)
print(mu)
# Do mean variance optimization using efficient frontier
rf_ef = EfficientFrontier(mu, S, weight_bounds=(0.05, 0.1))
rf_ef.min_volatility()
weights = rf_ef.clean_weights()
print(weights)

         AAPL         AMD        AMZN        CCJ        COST        GOOG   
0  177.326451  129.023400  131.336841  27.149476  530.545925  118.821504  \
1  175.984309  136.930700  131.703415  26.849531  534.203489  126.013539   
2  176.729634  135.176599  129.872910  25.827114  533.955277  125.632179   
3  176.844372  131.809300  130.689113  24.635293  536.731821  125.915529   
4  176.693935  128.331000  131.252893  25.059906  517.334901  125.606205   
5  176.831215  124.074900  127.158114  23.718244  528.963427  124.271205   
6  176.698693  120.069300  124.584219  24.192806  535.073554  124.457235   
7  176.698693  113.081301  127.025829  24.265536  539.103150  118.353554   
8  176.698693  111.371600  133.599519  23.228341  538.678189  124.554674   
9  176.698693  106.223600  127.327308  23.390912  542.005150  122.303634   

           GS         JPM         LLY        META        MSFT        NEE   
0  329.292164  139.587898  444.999999  277.523200  324.692267  75.595724  \
1  331.3523

In [137]:

future_prices = {}
for ticker in tickers:
    # Scale the input data
    scaler = MinMaxScaler()
    prices_x_reshaped = np.reshape(short_window_prices[ticker][0], (len(short_window_prices[ticker][0]), measurement_days))  # Reshape prices_x
    prices_x_scaled = scaler.fit_transform(prices_x_reshaped)
    prices_y_scaled = scaler.transform(np.reshape(short_window_prices[ticker][1], (-1, 1)))
    prices_test_scaled = scaler.transform(short_window_prices[ticker][2])

    # Create and train the MLPRegressor model
    model = MLPRegressor(hidden_layer_sizes=(64, 64), activation='relu', solver='adam', random_state=42)
    model.fit(prices_x_scaled, prices_y_scaled)

    # Predict future prices
    future_prices[ticker] = scaler.inverse_transform(model.predict(prices_test_scaled))

future_prices = pd.DataFrame(future_prices, index=range(measurement_days, measurement_days + remaining_days))
# future_prices = {}

# for ticker in tickers:
#     price_movements = np.array([1 if stock_data[i+measurement_days] > stock_data[i+measurement_days-1] else 0 for i in range(len(stock_data)-measurement_days)])
#     price_movements = price_movements[measurement_days:]

#     logreg = LogisticRegression()
#     logreg.fit(short_window_prices[ticker][0], price_movements)

#     future_price_movements = logreg.predict_proba(short_window_prices[ticker][2])[:, 1]
#     future_prices[ticker] = [stock_prices[ticker][-1] * (1 + movement) for movement in future_price_movements]


# future_prices = pd.DataFrame(future_prices)
# print(future_prices)

# Construct covariance matrix of future stock prices
# cov_matrix = risk_models.sample_cov(future_prices)
S = risk_models.CovarianceShrinkage(future_prices).ledoit_wolf()
# plotting.plot_covariance(S, plot_correlation=True)

# Use capm to find expected returns on future prices
mu = expected_returns.capm_return(future_prices)
print(mu)
# Do mean variance optimization using efficient frontier
logreg_ef = EfficientFrontier(mu, S, weight_bounds=(0.05, 0.1))
logreg_ef.min_volatility()
weights = logreg_ef.clean_weights()
print(weights)



ValueError: X has 1 features, but MinMaxScaler is expecting 10 features as input.

In [111]:
future_prices = {}

for ticker in tickers:
    svm = SVC(kernel='poly')
    svm.fit(short_window_prices[ticker][0], price_movements)
    
    future_price_movements = svm.predict(short_window_prices[ticker][2])
    future_prices[ticker] = [stock_prices[ticker][-1] * (1 + movement) for movement in future_price_movements]

future_prices = pd.DataFrame(future_prices)
# Construct covariance matrix of future stock prices
# cov_matrix = risk_models.sample_cov(future_prices)
S = risk_models.CovarianceShrinkage(future_prices).ledoit_wolf()
# plotting.plot_covariance(S, plot_correlation=True)

# Use capm to find expected returns on future prices
mu = expected_returns.capm_return(future_prices)
print(mu)
# Do mean variance optimization using efficient frontier
svm_ef = EfficientFrontier(mu, S, weight_bounds=(0.05, 0.1))
svm_ef.min_volatility()
weights = svm_ef.clean_weights()
print(weights)



AAPL    2.000000e-02
AMD     2.000000e-02
AMZN    2.000000e-02
CCJ     1.918831e+07
COST    1.569952e+07
GOOG    2.000000e-02
GS     -1.569952e+07
JPM     2.000000e-02
LLY     8.721957e+06
META    2.000000e-02
MSFT    4.535418e+07
NEE     2.000000e-02
PFE     2.000000e-02
SAP    -1.221074e+07
WMT     2.616587e+07
Name: mkt, dtype: float64
OrderedDict([('AAPL', 0.06833), ('AMD', 0.07172), ('AMZN', 0.06833), ('CCJ', 0.05), ('COST', 0.05), ('GOOG', 0.06833), ('GS', 0.1), ('JPM', 0.06833), ('LLY', 0.05), ('META', 0.06833), ('MSFT', 0.05), ('NEE', 0.06833), ('PFE', 0.06833), ('SAP', 0.1), ('WMT', 0.05)])


In [112]:
rf_ef.portfolio_performance(verbose=True)
print("\n")
logreg_ef.portfolio_performance(verbose=True)
print("\n")
svm_ef.portfolio_performance(verbose=True)
print("\n")

Expected annual return: -40.0%
Annual volatility: 7.7%
Sharpe Ratio: -5.44


Expected annual return: 6338987828062.7%
Annual volatility: 249.2%
Sharpe Ratio: 25434617140.36


Expected annual return: 296546535.1%
Annual volatility: 113.9%
Sharpe Ratio: 2604464.20




# Mean-Variance Optimization